In [11]:
@log
def add(a, b):
    """Adds two numbers.
    """
    return a + b

In [12]:
@log
def mul(a, b):
    """Multiplies two numbers.
    """
    return a * b

In [13]:
def log(func):
    def wrapper(*args):
        print("Calling {}.{}{}".format(func.__module__, func.__name__, args))
        r = func(*args)
        return r
    return wrapper

In [14]:
mul(12, 34)

Calling __main__.mul(12, 34)


408

In [15]:
add

<function __main__.log.<locals>.wrapper>

```
avg(10)=> 10
avg(20)=> 20
avg(30)=> 20
```

In [16]:
class Averager(object):
    def __init__(self):
        self.numbers = []
        
    def __call__(self, n):
        self.numbers.append(n)
        return sum(self.numbers) / len(self.numbers)

In [17]:
avg = Averager()

In [18]:
avg(10)

10.0

In [19]:
avg(20)

15.0

In [20]:
def averager():
    items = []
    
    def avg(n):
        items.append(n)
        return sum(items)/ len(items)
    return avg

In [21]:
avg = averager()

In [22]:
avg

<function __main__.averager.<locals>.avg>

In [25]:
avg.__code__.co_varnames

('n',)

In [26]:
avg.__code__.co_freevars

('items',)

In [27]:
avg(10)

10.0

In [28]:
avg(20)

15.0

In [29]:
def averager():
    total = 0
    count = 0
    
    def avg(n):
        total += n
        count += 1
        return total/count
    return avg

In [30]:
avg = averager()

In [31]:
avg(10)

UnboundLocalError: local variable 'total' referenced before assignment

In [32]:
def averager():
    total = 0
    count = 0
    
    def avg(n):
        nonlocal total
        nonlocal count
        total += n
        count += 1
        return total/count
    return avg

In [33]:
avg = averager()

In [34]:
avg(10)

10.0

In [35]:
avg.__code__.co_freevars

('count', 'total')

In [41]:
def log(func):
    def wrapper(*args, **kwargs):
        print("Calling {}.{}{}{}".format(func.__module__, func.__name__, args, kwargs))
        r = func(*args, **kwargs)
        return r
    return wrapper

In [42]:
@log
def hello(a, b, c=30):
    print(a, b, c)

In [43]:
hello(2, 3, c=9)

Calling __main__.hello(2, 3){'c': 9}
2 3 9


In [45]:
hello = log(hello)

In [46]:
hello

<function __main__.log.<locals>.wrapper>

In [47]:
hello.__code__.co_varnames

('args', 'kwargs', 'r')

In [48]:
hello.__code__.co_freevars

('func',)

In [60]:
import time
def clock(func):
    def wrapper(*args, **kwargs):
        start = time.perf_counter()
        r = func(*args, **kwargs)
        end = time.perf_counter()
        print('Total time: {}{} => {:.6f}'.format(func.__name__, args, end-start))
        return r
    return wrapper

In [61]:
@clock
def fact(n):
    return n if n == 1 else n * fact(n-1)

In [62]:
fact(20)

Total time: fact(1,) => 0.000001
Total time: fact(2,) => 0.000134
Total time: fact(3,) => 0.000182
Total time: fact(4,) => 0.000218
Total time: fact(5,) => 0.000256
Total time: fact(6,) => 0.000291
Total time: fact(7,) => 0.000324
Total time: fact(8,) => 0.000358
Total time: fact(9,) => 0.000390
Total time: fact(10,) => 0.000423
Total time: fact(11,) => 0.000747
Total time: fact(12,) => 0.000827
Total time: fact(13,) => 0.000871
Total time: fact(14,) => 0.000907
Total time: fact(15,) => 0.000944
Total time: fact(16,) => 0.000982
Total time: fact(17,) => 0.001017
Total time: fact(18,) => 0.001056
Total time: fact(19,) => 0.001090
Total time: fact(20,) => 0.001127


2432902008176640000

### Decorators with parameters 

In [72]:
filename = 'tracker.log'
def track(out=None):
    def log(func):
        def wrapper(*args):
            r = func(*args)
            msg = "Calling {}.{}{}\n".format(func.__module__, func.__name__, args)
            if not out:
                with open(filename, 'a') as f:
                    f.write(msg)
            else:
                out.write(msg)
            return r
        return wrapper
    return log

In [73]:
import sys

In [74]:
@track()
def add(a, b):
    return a + b

In [77]:
add(10, 20)

30

### class decorator

In [79]:
registry = {}

In [80]:
def register(cls):
    registry[cls.__clsid__] = cls
    return cls

In [82]:
@register
class Foo:
    __clsid__ = '1234'

In [83]:
registry

{'1234': __main__.Foo}

In [86]:
def singleton(cls):
    instances = {}
    
    def get_instance(*args, **kwargs):
        if cls not in instances:
            instances[cls] = cls(*args, **kwargs)
        return instances[cls]
    return get_instance

In [87]:
@singleton
class Foo:
    pass

In [88]:
f = Foo()

In [89]:
f.x = 10

In [90]:
g = Foo()

In [91]:
g.x

10

In [92]:
f

In [93]:
id(f)

4368445960

In [94]:
id(g)

4368445960

In [150]:
import functools

def timer(cls):
    class Wrapper(object):
        def __init__(self, *args, **kwargs):
            self.wrapper_obj = cls(*args, **kwargs)
            
        def __getattr__(self, attr):
            if callable(getattr(self.wrapper_obj, attr)):
                def myfunc(*args, **kwargs):
                    start = time.perf_counter()
                    func = getattr(self.wrapper_obj, attr)
                    r = func(*args, **kwargs)
                    total_time = time.perf_counter() - start
                    print('Total time: {}{}{} => {:.6f}'.format(func.__name__, args, kwargs, total_time))
                    return r
                return myfunc
            else:
                return getattr(self.wrapper_obj, attr)
    return Wrapper

In [151]:
@timer
class Calculator:
    """Simple Calculator
    """
    def add(self, a, b):
        return a + b
    
    def mul(self, a, b):
        return a * b

In [152]:
c = Calculator()

In [153]:
c

<__main__.timer.<locals>.Wrapper at 0x104713ba8>

In [154]:
c.add

<function __main__.timer.<locals>.Wrapper.__getattr__.<locals>.myfunc>

In [155]:
c.add(1,2)

Total time: add(1, 2){} => 0.000002


3

### Inbuilt decorators

In [162]:
class Sale:
    def __init__(self, name, date, count, price):
        self.name = name
        self.date = date
        self.count = int(count)
        self._price = float(price)
        
    @property
    def price(self):
        return self._price
    
    @price.setter
    def price(self, value):
        try:
            value = float(value)
        except ValueError:
            raise ValueError('Invalid value for price')
        else:
            self._price = value
    
    @classmethod
    def fromString(cls, s):
        return cls(*cls.cleanstr(s))
    
    @staticmethod
    def cleanstr(s):
        return [i.strip() for i in s.split()]
    
    @property
    def cost(self):
        return self.price * self.count

In [163]:
c = Sale.fromString('Apple 2012-12-23 34 45.23')

In [164]:
c

In [165]:
c.price

45.23

In [167]:
c.cost

1537.82

In [168]:
c.name

'Apple'

#### functools.wraps 

In [169]:
import functools

def log(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        print("Calling {}.{}{}{}".format(func.__module__, func.__name__, args, kwargs))
        r = func(*args, **kwargs)
        return r
    return wrapper

In [170]:
@log
def add(a, b):
    """Adds two numbers.
    """
    return a + b

In [171]:
help(add)

Help on function add in module __main__:

add(a, b)
    Adds two numbers.



In [172]:
add

<function __main__.add>

In [175]:
import functools

@functools.lru_cache()
@clock
def fact(n):
    return n if n == 1 else n * fact(n-1)

In [176]:
fact(10)

Total time: fact(1,) => 0.000001
Total time: fact(2,) => 0.000787
Total time: fact(3,) => 0.001014
Total time: fact(4,) => 0.001306
Total time: fact(5,) => 0.001490
Total time: fact(6,) => 0.001724
Total time: fact(7,) => 0.001890
Total time: fact(8,) => 0.002116
Total time: fact(9,) => 0.002319
Total time: fact(10,) => 0.002372


3628800

In [177]:
fact(12)

Total time: fact(11,) => 0.000002
Total time: fact(12,) => 0.000728


479001600

In [179]:
fact(15)

Total time: fact(13,) => 0.000002
Total time: fact(14,) => 0.000678
Total time: fact(15,) => 0.000744


1307674368000

In [180]:
# stacked decorators

In [181]:
def makebold(fn):
    def wrapped():
        return '<b>' + fn() + '</b>'
    return wrapped

In [182]:
def makeitalic(fn):
    def wrapped():
        return '<i>' + fn() + '</i>'
    return wrapped

In [188]:
# @makebold
# @makeitalic
def hello():
    return 'Hello!'

In [189]:
hello()

'Hello!'

In [190]:
hello = makebold(makeitalic(hello))

In [191]:
hello()

'<b><i>Hello!</i></b>'